In [21]:
import os
import numpy as np
import pandas as pd
import pandas_ta as ta
from pathlib import Path
from datetime import datetime


candlestick_frame = 60*4

downloads_path = str(Path.home() / "Downloads")
raw_data = f'{downloads_path}\\01-11-2019  15-08-2023.csv'
df = pd.read_csv(raw_data)
df['sma'] = ta.sma(df['close'], length=7)

ts = list(df['time'])
open = list(df['open'])
high = list(df['high'])
low = list(df['low'])
close = list(df['close'])
volume = list(df['volume'])
sma = list(df['sma'])


tot = len(ts)
long_runup_lst = []
long_drawdown_lst = []
short_runup_lst = []
short_drawdown_lst = []

for idx in range(tot):
    if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
        max_high = max(high[idx:idx+candlestick_frame])
        min_low = min(low[idx:idx+candlestick_frame])
        entry_price = open[idx]
        long_runup_lst.append(round((max_high*100/entry_price)-100, 2))
        long_drawdown_lst.append(round((min_low*100/entry_price)-100, 2))
        short_runup_lst.append(round((entry_price*100/min_low)-100, 2))
        short_drawdown_lst.append(round((entry_price*100/max_high)-100, 2))
    else:
        long_runup_lst.append(0)
        long_drawdown_lst.append(0)
        short_runup_lst.append(0)
        short_drawdown_lst.append(0)     


df['long_runup'] = long_runup_lst
df['long_drawdown'] = long_drawdown_lst
df['short_runup'] = short_runup_lst
df['short_drawdown'] = short_drawdown_lst

exp_df_fn = f'{downloads_path}\\exp_df.csv'
df.to_csv(exp_df_fn, index=False)
# os.system(f"start EXCEL.EXE {exp_df_fn}")


In [26]:
min(short_drawdown_lst)

-32.97